## Apply GA to the following problem
reference: https://applenob.github.io/ga.html

Target: f(x) = x + 10sin(5x) + 7cos(4x), get maximum from x in [0,10]

Set up: x in [0,10], split into 100000 pieces. 100000's in [2^16,2^17] so x is a vector of 17 binary digits.

In [118]:
import math
import random
import operator
import numpy as np

In [130]:
class GA_max():
    def __init__(self,length,count,range_x,target,retain_rate,random_select_rate,mutation_rate):
        #initial settings, get length,count and initial population
        self.length = length
        self.count = count
        self.range_x = range_x
        self.target= target
        self.retain_rate= retain_rate
        self.random_select_rate= random_select_rate
        self.mutation_rate= mutation_rate
        self.population = self.get_population(length,count)
        
    def get_population(self,length,count):
        return [self.get_chromesome(length) for i in range(count)]
    
    def get_chromesome(self,length):
        #generate random chromesome
        chromosome = 0
        for i in range(length):
            chromosome |= (1<<i) * random.randint(0, 1)
        return chromosome
    
    def decode(self,chromosome):
        #since chromosome is from 0-131071, we need to convert it to 0-10
        return 1.0 * chromosome * self.range_x / (2**self.length-1)
    
    
    def fit_target(self, chromosome,target):
        #feed teh chromesome into the target function
        x = self.decode(chromosome)
        return target(x)
    
    def selection(self, retain_rate, random_select_rate,target):
        #list all populations, sort
        clist = [(self.fit_target(chromosome,target), chromosome) for chromosome in self.population]
        clist = [x[1] for x in sorted(clist, reverse=True)]
        #select strong
        retain_length  = int(len(clist) * retain_rate)
        parents = clist[:retain_length]
        #select weak at random
        for chromosome in clist[retain_length:]:
            if random.random() < random_select_rate:
                parents.append(chromosome)
        return parents
    

    def crossover(self, parents):
       #generate children 
        children = []
        #count needed
        target_count = len(self.population) - len(parents)
        #when children is not enough, loop
        while len(children) < target_count:
            male = random.randint(0, len(parents)-1)
            female = random.randint(0, len(parents)-1)
            if male != female:
                cross_pos = random.randint(0, self.length)
                #mask first
                mask = 0
                for i in range(cross_pos):
                    mask |= (1 << i) 
                male = parents[male]
                female = parents[female]               
                # child get everything after pos from male and before from female
                child = ((male & mask) | (female & ~mask)) & ((1 << self.length) - 1)
                children.append(child)
        #The next generation is comprised of parents and children
        self.population = parents + children
        

    def mutation(self, rate):
        #randomly mutate one element in each chromosome
        for i in range(self.length):
            if random.random() < rate:
                j = random.randint(0, self.length-1)
                self.population[i] ^= 1 << j

    
    def evolve(self, retain_rate, random_select_rate, mutation_rate,target):
        parents = self.selection(retain_rate, random_select_rate,target)
        self.crossover(parents)
        self.mutation(mutation_rate)


    def result(self):
        graded = [(self.fit_target(chromosome,target), chromosome) for chromosome in self.population]
        graded = [x[1] for x in sorted(graded, reverse=True)]
        return self.decode(graded[0])
    

test

In [139]:
def target(x):
    return x + 10 * np.sin(5*x) + 7 * np.cos(4*x)

In [160]:
ga = GA_max(17, 200,10,target,0.2,0.5,0.05)

In [161]:
for x in range(200):
         ga.evolve(ga.retain_rate,ga.random_select_rate,ga.mutation_rate,ga.target)

In [162]:
ga.result()

7.856734136460392